## ***Step 1: Install Required Libraries***

In [ ]:
# Install necessary libraries for training and fine-tuning models

# `transformers` - Hugging Face library for working with pre-trained transformer models
!pip install -q transformers

# `datasets` - Library to easily access and load datasets from the Hugging Face hub
!pip install -q datasets

# `accelerate` - Optimizes training by utilizing multiple devices like GPUs/TPUs for faster computation
!pip install -q accelerate

# `bitsandbytes` - Provides efficient techniques for training large models with lower memory usage
!pip install -q bitsandbytes

# `peft` - Library offering efficient fine-tuning methods that minimize the number of trainable parameters
!pip install -q peft

# `sentencepiece` - Tokenizer library used for tokenization in models like T5 and BERT
!pip install -q sentencepiece

In [ ]:
# Load datasets from Hugging Face or local files
from datasets import load_dataset

# Automatically select the right tokenizer for the model
from transformers import AutoTokenizer

# Load a pre-trained model for sequence classification tasks
from transformers import AutoModelForSequenceClassification

## ***Step 2: Load Dataset and Model***

In [ ]:
# Load the IMDb dataset using the `load_dataset` function from the `datasets` library.
dataset = load_dataset("imdb")

In [ ]:
# Define the model name to be used for fine-tuning. We are using "bert-base-uncased",
model_name = "bert-base-uncased"

# Load the tokenizer associated with the BERT model. The tokenizer is responsible for converting raw text into tokens (numerical representations) that can be fed into the model.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained BERT model for sequence classification. 
# (positive or negative sentiment in the case of the IMDb dataset).
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## ***Step 3: Preprocess the Data***

In [ ]:
# Define a preprocessing function to tokenize the text
# It truncates and pads the text to a maximum length of 512 tokens
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply the preprocessing function to the dataset in batches
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## ***Step 4: Fine-Tune with LoRA (Low-Rank Adaptation)***

In [ ]:
# Import LoRA configuration and model utilities from `peft` library
from peft import LoraConfig, get_peft_model

In [ ]:
# Define LoRA (Low-Rank Adaptation) configuration
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices (controls the size of adaptation)
    lora_alpha=16,  # Scaling factor for the low-rank adaptation
    target_modules=["query", "value"],  # Modules of the model where LoRA will be applied
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    bias="none",  # Do not apply LoRA to the bias terms
    task_type="SEQ_CLS"  # The type of task (sequence classification in this case)
)


In [ ]:
# Apply LoRA (Low-Rank Adaptation) to the model using the defined configuration
# This modifies the model to incorporate LoRA, making it more efficient for fine-tuning
model = get_peft_model(model, lora_config)

## ***Step 5: Fine-Tune with QLoRA (Quantized LoRA)***

In [ ]:
# Import BitsAndBytesConfig for model quantization configuration (memory efficiency)
from transformers import BitsAndBytesConfig

# Import torch for tensor operations and model training with GPUs/CPUs
import torch

In [ ]:
# Define quantization configuration using BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load model with 4-bit precision for reduced memory usage
    bnb_4bit_use_double_quant=True,  # Enable double quantization for improved efficiency
    bnb_4bit_quant_type="nf4",  # Use 'nf4' quantization format for more accurate quantization
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 precision for computation
)

In [ ]:
# Load the pre-trained model with quantization configuration applied
# The model is loaded using 4-bit quantization for reduced memory usage and faster inference
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,  # The model name or path
    quantization_config=quantization_config,  # Apply the quantization settings
    num_labels=2  # Specify the number of labels for sequence classification (binary classification)
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## ***Step 6: Fine-Tune with PEFT (Parameter-Efficient Fine-Tuning)***

In [ ]:
# Import the necessary classes from the `peft` library
# `PeftModel` is used to wrap the model with Parameter Efficient Fine-Tuning (PEFT) techniques.
# `LoraConfig` is used to define the configuration for LoRA (Low-Rank Adaptation), a method to efficiently fine-tune models.
from peft import PeftModel, LoraConfig

In [ ]:
# Configure the LoRA (Low-Rank Adaptation) method for model fine-tuning
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices for adaptation
    lora_alpha=16,  # Scaling factor for the LoRA matrices
    lora_dropout=0.1,  # Dropout rate of 10% to prevent overfitting
    task_type="SEQ_CLS",  # Defines the task type as sequence classification (SEQ_CLS)
    inference_mode=False  # Sets the model in training mode, not inference mode
)

In [ ]:
# Apply LoRA PEFT (Parameter Efficient Fine-Tuning) to the model using the defined configuration
model = PeftModel(model, lora_config)  # Wraps the model with LoRA-based fine-tuning

## ***Step 7: Fine-Tune with SFT (Supervised Fine-Tuning)***

In [ ]:
# Import the Trainer class from the Hugging Face transformers library
from transformers import Trainer, TrainingArguments

In [ ]:
# Define training arguments with configuration options for training
training_args = TrainingArguments(
    output_dir="./results",  # Directory to store model and training results
    evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
    learning_rate=2e-5,  # Set learning rate for optimization
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    num_train_epochs=1,  # Number of training epochs
    weight_decay=0.01,  # Regularization to prevent overfitting
    save_strategy="epoch",  # Save model checkpoints at the end of each epoch
    logging_dir="./logs",  # Directory to store logs for tracking
    logging_steps=10,  # Log every 10 steps during training
    fp16=True,  # Enable mixed precision training to speed up training and reduce memory usage
)

# Create a Trainer instance that will handle the training loop
trainer = Trainer(
    model=model,  # Model to be trained
    args=training_args,  # Training configuration
    train_dataset=tokenized_dataset["train"],  # Training dataset
    eval_dataset=tokenized_dataset["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer used for processing input data
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-4cf45eb80476>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model using the previously defined trainer
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.387000,No log


TrainOutput(global_step=3125, training_loss=0.526418505859375, metrics={'train_runtime': 1038.5172, 'train_samples_per_second': 24.073, 'train_steps_per_second': 3.009, 'total_flos': 6600425625600000.0, 'train_loss': 0.526418505859375, 'epoch': 1.0})

## ***Step 8: Evaluate the Model***

In [ ]:
# Evaluate the model on the test dataset using the previously defined trainer
# This calculates metrics such as accuracy, loss, etc., on the test set
results = trainer.evaluate()

print(results)

{'eval_runtime': 305.7482, 'eval_samples_per_second': 81.767, 'eval_steps_per_second': 10.221, 'epoch': 1.0}


## ***Step 9: Save the Model***

In [ ]:
# Save the fine-tuned model to the specified directory
model.save_pretrained("./fine-tuned-model")

# Save the tokenizer associated with the fine-tuned model
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/vocab.txt',
 './fine-tuned-model/added_tokens.json',
 './fine-tuned-model/tokenizer.json')

## ***Step 10: Test the Fine-Tuned Model***

In [ ]:
# Import the pipeline function for easy model usage
from transformers import pipeline

# Create a text classification pipeline with the fine-tuned model and tokenizer
classifier = pipeline("text-classification", model="./fine-tuned-model", tokenizer=tokenizer)

# Test the classifier with a sample input text
sample_text = "I feel so happy today!"
result = classifier(sample_text)

# Print the classification result (label prediction)
print(result)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.6151294708251953}]


## ***Step 11: Save the Fine-Tuned Model to Local***

In [ ]:
# Import shutil for file operations and files for downloading in Google Colab
import shutil
from google.colab import files

# Specify the directory where the fine-tuned model is saved
model_directory = './fine-tuned-model'

# Create a .zip archive of the model directory for easy downloading
shutil.make_archive(model_directory, 'zip', model_directory)

# Trigger the download of the .zip file to your local machine
files.download(f'{model_directory}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>